In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

In [6]:
query = """
        SELECT * FROM Person
        
        """
q = pd.read_sql_query(query,conn)
print(q.shape)
q.head()

(37566, 4)


,index,PID,Name,Gender
0,0,nm0000288,Christian Bale,Male
1,1,nm0000949,Cate Blanchett,Female
2,2,nm1212722,Benedict Cumberbatch,Male
3,3,nm0365140,Naomie Harris,Female
4,4,nm0785227,Andy Serkis,Male


In [ ]:
#query2: List the names of all the actors who played in the movie 'Anand' (1971)

In [11]:
q2 =pd.read_sql_query("""
SELECT
 Distinct
 TRIM(P.Name) Actor_Name
FROM
 Movie M
 JOIN M_Cast MC
 ON M.MID = MC.MID
 JOIN Person p
 ON Trim(MC.PID) = Trim(P.PID)
WHERE
 M.title = 'Anand'
""", conn)
print(q2)

          Actor_Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
10    Brahm Bhardwaj
11      Gurnam Singh
12      Lalita Pawar
13       Durga Khote
14        Dara Singh
15     Johnny Walker
16         Moolchand


In [ ]:
#query3:List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 
1970 and > 1990.)

In [10]:
q3 = pd.read_sql_query("""
WITH
 ACTORS_BEFORE_1970 AS
 (
 SELECT
 DISTINCT
 TRIM(MC.PID) PID
 FROM
 Movie M
 JOIN M_Cast MC
 ON M.MID = MC.MID
 WHERE
 CAST(SUBSTR(M.year,-4) AS UNSIGNED) < 1970
 ),
 ACTORS_AFTER_1990 AS
 (
 SELECT
 DISTINCT
 TRIM(MC.PID) PID
 FROM
 Movie M
 JOIN M_Cast MC
 ON M.MID = MC.MID
 WHERE
 CAST(SUBSTR(M.year,-4) AS UNSIGNED) > 1990
 )
 SELECT
 DISTINCT
 TRIM(P.Name) Actor_Name
 FROM
 ACTORS_BEFORE_1970 A_1970
 JOIN ACTORS_AFTER_1990 A_1990
 ON A_1970.PID = A_1990.PID
 JOIN Person P
 ON A_1970.PID = TRIM(P.PID)
""", conn)
print(q3)


           Actor_Name
0        Rishi Kapoor
1    Amitabh Bachchan
2              Asrani
3        Zohra Sehgal
4     Parikshat Sahni
..                ...
295            Poonam
296     Jamila Massey
297       K.R. Vijaya
298             Sethi
299      Suryakantham

[300 rows x 1 columns]


In [ ]:
#query4:List all directors who directed 10 movies or more, in descending order of the number of movies they directed.Return the directors' names and the number of movies each of them directed

In [24]:
q4 = pd.read_sql_query("""
SELECT
 DISTINCT
 TRIM(P.NAME) DIRECTOR_NAME,
 Num.Number_Of_Movies_Directed
FROM
 (SELECT 
 PID,
 COUNT(MID) Number_Of_Movies_Directed
 FROM
 M_Director
 GROUP BY
 PID
 HAVING
 Number_Of_Movies_Directed>= 10
 ) Num
 JOIN PERSON P
 ON TRIM(Num.PID) = TRIM(P.PID)
ORDER BY 
 Num.Number_Of_Movies_Directed DESC
""", conn)
print(q4)

               DIRECTOR_NAME  Number_Of_Movies_Directed
0               David Dhawan                         39
1               Mahesh Bhatt                         35
2               Priyadarshan                         30
3            Ram Gopal Varma                         30
4               Vikram Bhatt                         29
5       Hrishikesh Mukherjee                         27
6                Yash Chopra                         21
7            Basu Chatterjee                         19
8             Shakti Samanta                         19
9               Subhash Ghai                         18
10             Shyam Benegal                         17
11  Abbas Alibhai Burmawalla                         17
12         Rama Rao Tatineni                         17
13            Manmohan Desai                         16
14                    Gulzar                         16
15              Raj N. Sippy                         16
16                Raj Kanwar                    

In [ ]:
#query6:Find the film(s) with the largest cast. Return the movie title and the size of the cast.

In [26]:
q6=pd.read_sql_query("""
WITH
 Cast AS
 (
 SELECT
 TRIM(MID) MID,
 COUNT(DISTINCT TRIM(PID)) Num_Of_Person
 FROM
 M_Cast
 GROUP BY 
 TRIM(MID) 
 )
 SELECT
 M.MID,
 M.title,
 C.Num_Of_Person
 FROM
 Cast C
 JOIN Movie M
 ON C.MID = TRIM(M.MID)
 WHERE
 C.Num_Of_Person= (
 SELECT
 MAX(Num_Of_Person)
     FROM
 Cast
 )
""", conn)
print(q6)

         MID          title  Num_Of_Person
0  tt5164214  Ocean's Eight            238


In [ ]:
#query1: List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year

In [22]:
 q1=pd.read_sql_query("""
WITH
 Comedy_Movies AS
 (
 SELECT
 -- COUNT(*) 107
 MG.MID
 FROM
 GENRE G
 JOIN M_GENRE MG
 ON G.GID = MG.GID
 WHERE
 TRIM(G.Name) = 'Comedy'
 ),
 Comedy_Movies_In_Leap_Year AS
 (
 SELECT
 M.MID,
 M.title,
 CAST(SUBSTR(M.year,-4) AS UNSIGNED) year
 FROM
 Comedy_Movies CM
 JOIN Movie M
 ON CM.MID = M.MID
 WHERE
 (CAST(SUBSTR(M.year,-4) AS UNSIGNED) %4 = 0 )
 )
 SELECT
 DISTINCT
 TRIM(P.Name) Director_Name,
 CM.title Movie_Name,
 CM.year
 FROM
 Comedy_Movies_In_Leap_Year CM
 JOIN M_Director MD
 ON CM.MID = MD.MID
 JOIN Person P
 ON TRIM(MD.PID) = TRIM(P.PID)
 ORDER BY 
 year
""", conn)
print(q1)

            Director_Name                        Movie_Name  year
0         Pankaj Parashar                    Ab Ayega Mazaa  1984
1            Kawal Sharma                         Maalamaal  1988
2            Jabbar Patel                  Ek Hota Vidushak  1992
3            Mahesh Bhatt                   Papa Kahte Hain  1996
4               Bhagyaraj                       Mr. Bechara  1996
5                  Sachin              Navra Mazha Navsacha  2004
6            Govind Menon                 Kis Kis Ki Kismat  2004
7   Siddharth Anand Kumar                       Let's Enjoy  2004
8             Raj Kaushal                  Shaadi Ka Laddoo  2004
9       Srinivas Bhashyam                      Paisa Vasool  2004
10     Jagdish Rajpurohit                            Bumboo  2012
11           Anand Balraj          Daal Mein Kuch Kaala Hai  2012
12           Karan Razdan               Mr Bhatti on Chutti  2012
13         Rajnish Thakur  Mere Dost Picture Abhi Baaki Hai  2012
14       V

In [ ]:
#query5:

In [29]:
q5 = pd.read_sql_query("""
WITH
 MOVIES_WITH_NON_FEMALES AS
 (
 SELECT
 DISTINCT
 TRIM(MC.MID) MID
 FROM
 M_Cast MC
 JOIN Person P
 ON TRIM(MC.PID) = TRIM(P.PID)
 WHERE
 TRIM(P.Gender) IN ('Male','None') -- Considering None as not female.
 )
 SELECT
 CAST(SUBSTR(M.year,-4) AS UNSIGNED) year,
 COUNT(DISTINCT TRIM(MID) ) NUM_OF_MOV_WITH_ONLY_FEMALES
 FROM
 Movie M
 WHERE
 TRIM(MID) NOT IN (SELECT MID FROM MOVIES_WITH_NON_FEMALES)
 GROUP BY 
 CAST(SUBSTR(M.year,-4) AS UNSIGNED)
 ORDER BY
 year
""", conn)
print(q5)


   year  NUM_OF_MOV_WITH_ONLY_FEMALES
0  1939                             1
1  1999                             1
2  2000                             1
3  2009                             1
4  2012                             1
5  2018                             2
